In [ ]:
import numpy as np

from sltns.experiment._exp import ExperimentRunner

In [2]:
param_grid = {
    "prior_len": [250, 200, 120, 80, 60, 50, 40, 35, 25, 15],
    "lambda_1": [10, 1, 1e-1, 1e-3, 1e-4],
    "lambda_2": [1, 5, 10, 30, 50, 100, 500, 1000, 5000],
    "lambda_3": [0.1, 1, 10, 20, 30, 50, 100, 200, 500, 1000],
    "eta": [1e-3, 5e-3, 1e-2, 1e-4],
    "pq_norm": ["l1", "l2"],
}
fixed_kwargs = {
    "delta": 1e-3,
    "max_outer": 500,
    "max_inner": 50,
    "verbose": False,
    "patience": 3,
    "tol_inner": 1e-3,
}

In [ ]:
runner = ExperimentRunner(
    param_grid, fixed_kwargs, arima_orders=[(1, 0, 0)], verbose=False
)
seeds = runner.generate_seeds(50, master_seed=0)

val_sizes = [10, 30, 50]
test_sizes = [10, 20]

for w_val in val_sizes:
    for w_test in test_sizes:
        rmse_vals = []
        for seed in seeds:
            _series, X_tr, y_tr, X_va, y_va, X_te, y_te = runner.simulate_and_split(
                process_cfg={"kind": "ar1", "sigma": 1},
                seed=seed,
                series_length=500,
                w_val=w_val,
                w_test=w_test,
            )
            _, rmse = runner.evaluate_arima(y_train=y_tr, y_val=y_va, y_test=y_te)
            if not np.isnan(rmse):
                rmse_vals.append(rmse)

        avg_rmse = np.mean(rmse_vals) if rmse_vals else np.nan
        print(
            f"w_val={w_val:2d}, w_test={w_test:2d} → "
            f"Average RMSE over {len(rmse_vals)} seeds: {avg_rmse:.4f}"
        )

w_val=10, w_test=10 → Average RMSE over 50 seeds: 1.3059
w_val=10, w_test=20 → Average RMSE over 50 seeds: 1.3615
w_val=30, w_test=10 → Average RMSE over 50 seeds: 1.3059
w_val=30, w_test=20 → Average RMSE over 50 seeds: 1.3615
w_val=50, w_test=10 → Average RMSE over 50 seeds: 1.3059
w_val=50, w_test=20 → Average RMSE over 50 seeds: 1.3615


In [4]:
runner = ExperimentRunner(param_grid, fixed_kwargs)
seeds = runner.generate_seeds(50, master_seed=0)

df_all, summary_all, summary = runner.run_experiment_grid(
    process_cfgs=[{"kind": "random_walk", "sigma": 1}, {"kind": "ar1", "sigma": 1}],
    seeds=seeds,
    w_val=10,
    w_test=10,
    n_iter=100,
    series_length=500,
    full_refit=True,
    slice_prior=False,
)

summary

,kind,model,rmse_mean,rmse_std
0,ar1,ARIMA,1.720352,1.288973
1,ar1,MDRIFT abs,6.316093,5.312036
2,ar1,MDRIFT,1.554542,0.711626
3,ar1,Mean,1.364581,0.494876
4,ar1,OLS,1.366596,0.475112
5,random_walk,ARIMA,2.845598,1.894757
6,random_walk,MDRIFT abs,9.170585,9.045510
7,random_walk,MDRIFT,3.043513,1.618554
8,random_walk,Mean,8.828008,7.610204
9,random_walk,OLS,8.308415,4.625980


In [3]:
runner = ExperimentRunner(param_grid, fixed_kwargs)
seeds = runner.generate_seeds(50, master_seed=0)

df_all, summary_all, summary = runner.run_experiment_grid(
    process_cfgs=[{"kind": "random_walk", "sigma": 1}, {"kind": "ar1", "sigma": 1}],
    seeds=seeds,
    w_val=30,
    w_test=10,
    n_iter=100,
    series_length=500,
    full_refit=True,
    slice_prior=False,
)

summary

,kind,model,rmse_mean,rmse_std
0,ar1,ARIMA,1.378117,0.521862
1,ar1,MDRIFT abs,6.944992,7.603874
2,ar1,MDRIFT,1.557674,0.746167
3,ar1,Mean,1.364581,0.494876
4,ar1,OLS,1.366596,0.475112
5,random_walk,ARIMA,2.818738,1.964087
6,random_walk,MDRIFT abs,8.389333,9.513519
7,random_walk,MDRIFT,3.158676,1.649683
8,random_walk,Mean,8.828008,7.610204
9,random_walk,OLS,8.308415,4.625980


In [5]:
runner = ExperimentRunner(param_grid, fixed_kwargs)
seeds = runner.generate_seeds(50, master_seed=0)

df_all, summary_all, summary = runner.run_experiment_grid(
    process_cfgs=[{"kind": "random_walk", "sigma": 1}, {"kind": "ar1", "sigma": 1}],
    seeds=seeds,
    w_val=50,
    w_test=10,
    n_iter=100,
    series_length=500,
    full_refit=True,
    slice_prior=False,
)

summary

,kind,model,rmse_mean,rmse_std
0,ar1,ARIMA,1.379725,0.586670
1,ar1,MDRIFT abs,9.844382,13.318566
2,ar1,MDRIFT,1.526586,0.661173
3,ar1,Mean,1.364581,0.494876
4,ar1,OLS,1.366596,0.475112
5,random_walk,ARIMA,3.209296,3.553493
6,random_walk,MDRIFT abs,10.482858,11.125914
7,random_walk,MDRIFT,3.476157,2.814662
8,random_walk,Mean,8.828008,7.610204
9,random_walk,OLS,8.308415,4.625980


In [ ]:
# experiment with prior slicing
runner = ExperimentRunner(param_grid, fixed_kwargs)
seeds = runner.generate_seeds(50, master_seed=0)

df_all, summary_all, summary = runner.run_experiment_grid(
    process_cfgs=[{"kind": "random_walk", "sigma": 1}, {"kind": "ar1", "sigma": 1}],
    seeds=seeds,
    w_val=30,
    w_test=10,
    n_iter=100,
    series_length=500,
    full_refit=True,
    slice_prior=True,
)

summary

,kind,model,rmse_mean,rmse_std
0,ar1,ARIMA,1.378117,0.521862
1,ar1,MDRIFT abs,10.185233,18.560170
2,ar1,MDRIFT,1.514935,0.693652
3,ar1,Mean,1.364581,0.494876
4,ar1,OLS,1.366596,0.475112
5,random_walk,ARIMA,2.818738,1.964087
6,random_walk,MDRIFT abs,15.089652,20.091361
7,random_walk,MDRIFT,3.341663,1.922826
8,random_walk,Mean,8.828008,7.610204
9,random_walk,OLS,8.308415,4.625980


In [8]:
runner = ExperimentRunner(param_grid, fixed_kwargs)
seeds = runner.generate_seeds(50, master_seed=0)

df_all, summary_all, summary = runner.run_experiment_grid(
    process_cfgs=[{"kind": "random_walk", "sigma": 1}, {"kind": "ar1", "sigma": 1}],
    seeds=seeds,
    w_val=10,
    w_test=10,
    n_iter=100,
    series_length=500,
    full_refit=False,
    slice_prior=False,
)

summary

,kind,model,rmse_mean,rmse_std
0,ar1,ARIMA,1.720352,1.288973
1,ar1,MDRIFT abs,1.602941,0.734482
2,ar1,MDRIFT,1.503251,0.721155
3,ar1,Mean,1.364581,0.494876
4,ar1,OLS,1.366596,0.475112
5,random_walk,ARIMA,2.845598,1.894757
6,random_walk,MDRIFT abs,2.730443,1.705719
7,random_walk,MDRIFT,3.102228,1.904325
8,random_walk,Mean,8.828008,7.610204
9,random_walk,OLS,8.308415,4.625980


In [6]:
runner = ExperimentRunner(param_grid, fixed_kwargs)
seeds = runner.generate_seeds(50, master_seed=0)

df_all, summary_all, summary = runner.run_experiment_grid(
    process_cfgs=[{"kind": "random_walk", "sigma": 1}, {"kind": "ar1", "sigma": 1}],
    seeds=seeds,
    w_val=30,
    w_test=10,
    n_iter=100,
    series_length=500,
    full_refit=False,
    slice_prior=False,
)

summary

,kind,model,rmse_mean,rmse_std
0,ar1,ARIMA,1.378117,0.521862
1,ar1,MDRIFT abs,1.442531,0.513023
2,ar1,MDRIFT,1.430331,0.498870
3,ar1,Mean,1.364581,0.494876
4,ar1,OLS,1.366596,0.475112
5,random_walk,ARIMA,2.818738,1.964087
6,random_walk,MDRIFT abs,3.143213,1.787135
7,random_walk,MDRIFT,3.673212,2.482769
8,random_walk,Mean,8.828008,7.610204
9,random_walk,OLS,8.308415,4.625980


In [7]:
runner = ExperimentRunner(param_grid, fixed_kwargs)
seeds = runner.generate_seeds(50, master_seed=0)

df_all, summary_all, summary = runner.run_experiment_grid(
    process_cfgs=[{"kind": "random_walk", "sigma": 1}, {"kind": "ar1", "sigma": 1}],
    seeds=seeds,
    w_val=50,
    w_test=10,
    n_iter=100,
    series_length=500,
    full_refit=False,
    slice_prior=False,
)

summary

,kind,model,rmse_mean,rmse_std
0,ar1,ARIMA,1.379725,0.586670
1,ar1,MDRIFT abs,1.427789,0.514460
2,ar1,MDRIFT,1.397056,0.533880
3,ar1,Mean,1.364581,0.494876
4,ar1,OLS,1.366596,0.475112
5,random_walk,ARIMA,3.209296,3.553493
6,random_walk,MDRIFT abs,4.108764,2.478094
7,random_walk,MDRIFT,4.674620,3.427828
8,random_walk,Mean,8.828008,7.610204
9,random_walk,OLS,8.308415,4.625980


In [9]:
runner = ExperimentRunner(param_grid, fixed_kwargs)
seeds = runner.generate_seeds(50, master_seed=0)

df_all, summary_all, summary = runner.run_experiment_grid(
    process_cfgs=[{"kind": "random_walk", "sigma": 1}, {"kind": "ar1", "sigma": 1}],
    seeds=seeds,
    w_val=10,
    w_test=20,
    n_iter=100,
    series_length=500,
    full_refit=True,
    slice_prior=False,
)

summary

,kind,model,rmse_mean,rmse_std
0,ar1,ARIMA,2.554750,3.666856
1,ar1,MDRIFT abs,10.853581,14.109113
2,ar1,MDRIFT,1.641889,0.699640
3,ar1,Mean,1.384585,0.359281
4,ar1,OLS,1.410015,0.358284
5,random_walk,ARIMA,5.299323,5.840411
6,random_walk,MDRIFT abs,10.344391,9.514125
7,random_walk,MDRIFT,3.568436,2.043701
8,random_walk,Mean,9.059587,7.543761
9,random_walk,OLS,8.598083,4.623588


In [10]:
runner = ExperimentRunner(param_grid, fixed_kwargs)
seeds = runner.generate_seeds(50, master_seed=0)

df_all, summary_all, summary = runner.run_experiment_grid(
    process_cfgs=[{"kind": "random_walk", "sigma": 1}, {"kind": "ar1", "sigma": 1}],
    seeds=seeds,
    w_val=30,
    w_test=20,
    n_iter=100,
    series_length=500,
    full_refit=True,
    slice_prior=False,
)

summary

,kind,model,rmse_mean,rmse_std
0,ar1,ARIMA,2.031156,2.677797
1,ar1,MDRIFT abs,8.889384,8.451673
2,ar1,MDRIFT,1.733542,0.769416
3,ar1,Mean,1.384585,0.359281
4,ar1,OLS,1.410015,0.358284
5,random_walk,ARIMA,5.043502,5.093016
6,random_walk,MDRIFT abs,12.304199,14.825632
7,random_walk,MDRIFT,3.702506,2.224326
8,random_walk,Mean,9.059587,7.543761
9,random_walk,OLS,8.598083,4.623588


In [11]:
runner = ExperimentRunner(param_grid, fixed_kwargs)
seeds = runner.generate_seeds(50, master_seed=0)

df_all, summary_all, summary = runner.run_experiment_grid(
    process_cfgs=[{"kind": "random_walk", "sigma": 1}, {"kind": "ar1", "sigma": 1}],
    seeds=seeds,
    w_val=50,
    w_test=20,
    n_iter=100,
    series_length=500,
    full_refit=True,
    slice_prior=False,
)

summary

,kind,model,rmse_mean,rmse_std
0,ar1,ARIMA,1.663793,1.444987
1,ar1,MDRIFT abs,10.083681,13.284642
2,ar1,MDRIFT,1.763033,1.207781
3,ar1,Mean,1.384585,0.359281
4,ar1,OLS,1.410015,0.358284
5,random_walk,ARIMA,4.757746,4.654491
6,random_walk,MDRIFT abs,11.015320,11.982001
7,random_walk,MDRIFT,4.219005,2.865538
8,random_walk,Mean,9.059587,7.543761
9,random_walk,OLS,8.598083,4.623588
